In [1]:

import sys
sys.path.append('/home/techt/Desktop/a4s')


### Step 1: Initial Data Loading

In [2]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Sickle_Cell_Anemia/GSE16728'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Characterization of whole blood gene expression profiles in sickle-cell disease patients using globin mRNA reduction"
!Series_summary	"Room temperature whole blood mRNA stabilization procedures, such as the PAX gene system, are critical for the application of transcriptional analysis to population-based clinical studies. Global transcriptome analysis of whole blood RNA using microarrays has proven to be challenging due to the high abundance of globin transcripts that constitute 70% of whole blood mRNA in the blood. This is a particular problem in patients with sickle-cell disease, secondary to the high abundance of globin-expressing nucleated red blood cells and reticulocytes in the circulation .  In order to more accurately measure the steady state whole blood transcriptome in sickle-cell patients, we evaluated the efficacy of reducing globin transcripts in PAXgene stabilized RNA samples for genome-wide transcriptome analyses using oligonucleotid

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_age = None  # define the functions when applicable

# Step 1: Determine if gene expression data is available
is_gene_available = True  # This dataset is about global transcriptome analysis using oligonucleotide arrays, so gene expression data is available.

# Step 2: Variable Availability and Data Type Conversion
sample_characteristics = {0: ['tissue: peripheral blood'],
                          1: ['subject: Control', 'subject: sickle-cell patient'], 
                          2: ['rna prep: PAXgene Globin reduced', 'rna prep: PAXgene', 'rna prep: PBMC'], 
                          3: ['molecule: Globin-depleted RNA', 'molecule: total RNA', 'molecule: RNA extracted from PBMC']}

# Determine keys for the variables
trait_row = 1  # 'Sickle_Cell_Anemia' corresponds to 'subject: Control' or 'subject: sickle-cell patient'
# There are no indications of age and gender data in this sample characteristics dictionary
age_row = gender_row = None 

# Data type conversion functions
def convert_trait(value):
    label = value.split(': ')[1]
    if label == 'Control':
        return 0
    elif label == 'sickle-cell patient':
        return 1
    else:
        return None

def convert_age(value):
    return None  # No age data available

def convert_gender(value):
    return None  # No gender data available

# Step 3: Save Metadata
save_cohort_info('GSE16728', './preprocessed/Sickle_Cell_Anemia/cohort_info.json', is_gene_available, trait_row is not None)

# Step 4: Clinical Feature Extraction
if trait_row is not None:
    selected_clinical_data = geo_select_clinical_features(clinical_data, 'Sickle_Cell_Anemia', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
    csv_path = './preprocessed/Sickle_Cell_Anemia/trait_data/GSE16728.csv'
    selected_clinical_data.to_csv(csv_path)
    print(preview_df(selected_clinical_data))


{'GSM419145': [0], 'GSM419146': [0], 'GSM419147': [0], 'GSM419148': [0], 'GSM419149': [0], 'GSM419150': [1], 'GSM419151': [1], 'GSM419152': [1], 'GSM419153': [1], 'GSM419154': [1], 'GSM419155': [0], 'GSM419156': [0], 'GSM419157': [0], 'GSM419158': [0], 'GSM419159': [0], 'GSM419160': [1], 'GSM419161': [1], 'GSM419162': [1], 'GSM419163': [1], 'GSM419164': [1], 'GSM419165': [0], 'GSM419166': [0], 'GSM419167': [0], 'GSM419168': [0], 'GSM419169': [0], 'GSM419170': [1], 'GSM419171': [1], 'GSM419172': [1], 'GSM419173': [1], 'GSM419174': [1]}


### Step 3: Gene Data Extraction

In [4]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row ids for the following step.
print(gene_data.index[:20])


Index(['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at', '1294_at',
       '1316_at', '1320_at', '1405_i_at', '1431_at', '1438_at', '1487_at',
       '1494_f_at', '1552256_a_at', '1552257_a_at', '1552258_at', '1552261_at',
       '1552263_at', '1552264_a_at', '1552266_at'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
requires_gene_mapping = True


### Step 5: Gene Annotation (Conditional)

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the soft file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at'], 'GB_ACC': ['U48705', 'M87338', 'X51757', 'X69699', 'L36861'], 'SPOT_ID': [nan, nan, nan, nan, nan], 'Species Scientific Name': ['Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens'], 'Annotation Date': ['Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014'], 'Sequence Type': ['Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence'], 'Sequence Source': ['Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Database'], 'Target Description': ['U48705 /FEATURE=mRNA /DEFINITION=HSU48705 Human receptor tyrosine kinase DDR gene, complete cds', 'M87338 /FEATURE= /DEFINITION=HUMA1SBU Human replication factor C, 40-kDa subunit (A1) mRNA, complete cds', "X51757 /FEATURE=cds /DEFINITION=HSP70B Human heat-shock protein HSP70B' gene", 'X69699 /FEATURE= /DEF

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Determine the keys for identifier and gene symbols from gene annotation
identifier_key = 'ID'
gene_symbol_key = 'Gene Symbol'

# 2. Get the dataframe storing the mapping between probe IDs and genes using the 'get_gene_mapping' function from the library
gene_mapping_df = get_gene_mapping(gene_annotation, identifier_key, gene_symbol_key)

# 3. Apply the mapping with the 'apply_gene_mapping' function from the library, and name the resulting gene expression dataframe "gene_data"
gene_data = apply_gene_mapping(gene_data, gene_mapping_df)


### Step 7: Data Normalization and Merging

In [8]:
# 1. Normalize the obtained gene data with the 'normalize_gene_symbols_in_index' function from the library.
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
gene_csv_path = './preprocessed/Sickle_Cell_Anemia/gene_data/GSE16728.csv'
normalized_gene_data.to_csv(gene_csv_path)

# 2. Merge the clinical and genetic data with the 'geo_merge_clinical_genetic_data' function from the library.
merged_data = geo_merge_clinical_genetic_data(selected_clinical_data, normalized_gene_data)

# 3. Determine whether the trait and some demographic attributes in the data is severely biased, and remove biased attributes.
trait_biased, unbiased_merged_data = judge_and_remove_biased_features(merged_data, 'Sickle_Cell_Anemia')

# If the trait is not severely biased, save the cohort information and the merged data.

# 4. Save the cohort information.
save_cohort_info('GSE16728', './preprocessed/Sickle_Cell_Anemia/cohort_info.json', True, True, trait_biased, merged_data)

if not trait_biased:
    # 5. If the trait is not severely biased, save the merged data to a csv file.
    csv_path = './preprocessed/Sickle_Cell_Anemia/GSE16728.csv'
    unbiased_merged_data.to_csv(csv_path)


For the feature 'Sickle_Cell_Anemia', the least common label is '0.0' with 15 occurrences. This represents 50.00% of the dataset.
The distribution of the feature 'Sickle_Cell_Anemia' in this dataset is fine.

